# 기사 URL이 주어졌을 때 본문의 단어 발췌하기

In [1]:
import requests
import re
from konlpy.tag import Mecab
from bs4 import BeautifulSoup as bs

1. Daum뉴스의 class='news_view'인 div태그에 본문 내용이 포함되어 있음을 확인
2. 크롤링 과정에서 제거해야 할 정보들 확인
 - 기자의 이메일(당연히 @를 포함하는 문자열로 식별가능)
 - 괄호에 둘러쌓여 부수적인 정보를 제공하는 기타 내용들(신문사 이름 등)
 
URL이 아래와 같은 기사의 본문에 포함된 단어들을 추출해 보려고 한다.

In [2]:
response = requests.get("https://news.v.daum.net/v/20200828203011138")
parsed = bs(response.text, 'html.parser')

참고로 이런 제목의 기사다.

In [3]:
print(parsed.select('div.head_view h3')[0].get_text())

코로나에 '돌봄 멘붕'.. 민주당 "가족돌봄휴가 10→30일 연장 추진"


기사의 본문 부분(body)에서 내용(text)을 추출하는 코드다.

In [4]:
body = parsed.find('div', attrs={'class':'news_view'})
text = body.find_all('p', attrs={'dmcf-ptype':'general'})

각 태그에서 텍스트를 추출하는 동시에 기자의 이메일과 기자의 이름을 제거하는 코드다. '홍길동 기자'와 같은 줄은 제거하기 위해 글자 수가 9 이상인 줄만 선택했다.

In [5]:
extracted = ' '.join([tag.get_text().strip() 
                      for tag in text 
                      if '@' not in tag.get_text() and len(tag.get_text()) > 8])

괄호로 싸인 문자열을 제거하는 과정에서 괄호에 싸인 글자 수를 최대 20까지로 지정했다. 글자 수에 제한을 두지 않으면 기사 초반에 있는 여는 괄호가 기사 말미에 있는 닫는 괄호와 한 쌍인 것으로 인식되서 기사 전체가 다 사라지는 불상사가 발생할 수 있는데, 이를 방지하기 글자 수를 지정했다. 무엇보다 20자가 넘어가는 정보는 부수적인 내용이 아닐 가능성이 크다. 결과를 확인하면 아래와 같다.

In [6]:
pattern = re.compile("[\[(].{1,20}[\])]")
print(pattern.sub('', extracted))

김태년 더불어민주당 원내대표는 28일 신종 코로나바이러스 감염증으로 인한 돌봄 공백 해소를 위해 가족돌봄휴가 기간 연장을 즉각 추진하겠다고 밝혔다. 김 원내대표는 이날 입장문을 내고 “코로나19 재확산으로 수도권 학교가 전면 원격수업에 돌입하고 유치원 휴원도 무기한 연장되며 돌봄 공백에 대한 우려가 높다”며 이같이 말했다. 그는 “특히 직장인 학부모들의 자녀 돌봄에 대한 대책이 시급한 상황”이라며 “직장인 학부모는 가족돌봄휴가 제도를 자녀의 원격수업, 등교 연기 등의 경우에도 사용할 수 있지만 문제는 휴가 한도 10일을 이미 다 쓴 가정이 많다는 것”이라고 추진 배경을 설명했다. 그러면서 "코로나19 같은 위기 상황에서는 휴가 기간을 20~30일까지 늘리도록 남녀고용평등법을 개정하겠다"고 했다. 김 원내대표는 “국회의장, 양당 원내대표 회동에서 코로나 관련 법안은 여야가 합의할 경우 상임위 숙려기간을 두지 않고 우선 처리하기로 합의했다”며 “개정안이 다음주라도 국회 본회의를 통과할 수 있도록 야당과의 협의를 신속하게 진행하겠다”고 말했다. 이어 “정부에서 근로자에게 최대 10일 간 지급하는 가족돌봄휴가 지원금의 지급기간 확대도 정부와 신속하게 협의하겠다”고 했다. 최근 문재인 대통령도 코로나19로 인한 돌볼 대책을 세울 것을 관계 부처에 지시했다.


## `koNLPy`를 통한 문장성분 추출

마지막으로 LDA의 입력 데이터인 단어를 얻기 위해 `koNLPy`에서 제공하는 `Mecab`형태소 분석기를 사용해 기사의 명사들을 추출했다. 

In [8]:
mecab = Mecab()
print([word for word in mecab.nouns(pattern.sub('', extracted)) if len(word) > 1])

['김태년', '민주당', '원내대표', '신종', '코로나', '바이러스', '감염증', '공백', '해소', '가족', '휴가', '기간', '연장', '추진', '원내대표', '이날', '입장', '코로나', '확산', '수도', '학교', '전면', '원격', '수업', '돌입', '유치원', '원도', '무기', '연장', '공백', '우려', '직장', '학부모', '자녀', '대책', '상황', '직장', '학부모', '가족', '휴가', '제도', '자녀', '원격', '수업', '등교', '연기', '경우', '사용', '문제', '휴가', '한도', '가정', '추진', '배경', '설명', '코로나', '위기', '상황', '휴가', '기간', '남녀', '고용', '평등', '개정', '원내대표', '국회', '의장', '양당', '원내대표', '회동', '코로나', '관련', '법안', '여야', '합의', '경우', '상임위', '숙려', '기간', '처리', '합의', '개정안', '다음', '주라', '국회', '본회의', '통과', '야당', '협의', '신속', '진행', '정부', '근로자', '최대', '지급', '가족', '휴가', '지원금', '지급', '기간', '확대', '정부', '신속', '협의', '최근', '문재', '대통령', '코로나', '대책', '관계', '부처', '지시']
